# 0. what for
1. 对大数据集进行清洗
# 1. Preparations
## 1.1 全局设置

In [65]:
#导入pandas包
import pandas as pd
import numpy as np
from tqdm import tqdm

auctions_selected_path = r'E:\DATA\large_dta\auctions_selected.csv' # 暂存data，只包含了必要的cols
PA_auction_id_path = r'E:\DATA\large_dta\PA_auction_id.csv'         # 暂存auction_id，是那些通过了规则筛选的auction id
auctions_cleaned_path = r'E:\DATA\large_dta\auctions_cleaned.csv'   # retail修正的data

# settings path
settings_GT_path = r'E:\DATA\large_dta\settings_GT.csv'
settings_NN_path = r'E:\DATA\large_dta\settings_NN.csv'

- read in

## 1.2 可以用的columns重新save data
1. 'k','y'分别表示bid inc和bid fee
2. adjvalue是amazon价格，worthupto是PA网站给出的零售价，
    - 这篇paper里用作
3. data中的columns包括:`['Unnamed: 0', 'auctionid', 'endtime', 'item', 'worthupto', 'adjvalue',
        'endbidlevel', 'endbidlevel_norm', 'endtimeperiod', 'shipping_swp',
        'shipping_am', 'beginnerauction', 'nailbiter', 'percentoff',
        'fixedprice', 'pennyauction', 'username', 'profit', 'profitperc',
        'at_only', 'us_only', 'amazon_match', 'type', 'worthupto_de',
        'currency', 'worthupto_com', 'exchangerate', 'cash', 'freebids',
        'consumer', 'y', 'k']`
4. data_2中的columns包括：`['Unnamed: 0', 'auctionid', 'item', 'adjvalue', 'nailbiter', 'type', 'k',
       'time', 'netvalue', 'adjvalue_ln', 'netvalue_ln', 'sunkcosts_agg',
       'sunkcosts_agg_ln', 'end', 'hour', 'week', 'day', 'dayofweek',
       'valuebin']`
5. 新保存的data后缀为`selected`，**实际上我们只需要data这一个auction-level的table就够了**

In [92]:

# name=pd.read_stata(r'E:\DATA\large_dta\auctions.dta')
# name.to_csv(r'E:\DATA\large_dta\auctions.csv', encoding="utf-8",header=True,index = False)
# print('\n')
# print('转换成功...')




转换成功...


In [8]:
col = ['auctionid',
    'item',
    'pennyauction','fixedprice','type',
    'currency','exchangerate',
    'adjvalue','worthupto',
    'endtimeperiod','k','y']
col_2 = ['auctionid','item',
    'type',
    'adjvalue',
    'time','end',
    'k']

data = pd.read_csv(r'E:\DATA\large_dta\auctions.csv')

In [9]:
data.auctionid.nunique() == data.shape[0]

True

In [10]:
data_tmp = data[col].copy()
data_tmp['k'] = data_tmp.k*0.01
data_tmp.to_csv(auctions_selected_path, encoding = 'utf-8', index=False, header=True)
# data_tmp.head()
del(data_tmp)
del(data)

## 1.3 ReRead
1. 重新读入数据，只包括需要的columns

In [11]:
data = pd.read_csv(auctions_selected_path, encoding = 'utf-8')
data.head()

,auctionid,item,pennyauction,fixedprice,type,currency,exchangerate,adjvalue,worthupto,endtimeperiod,k,y
0,144359,KOSMETIK BüRSTEN SET 6 TEILIG AUS HOLZ,0,0,Consumer Goods,Euro,1.2918,8.822994,9.029682,2.0,0.15,0.1
1,154753,KOSMETIK BüRSTEN SET 6 TEILIG AUS HOLZ,0,0,Consumer Goods,Euro,1.2774,8.929026,8.929026,1.0,0.15,0.1
2,147974,KOSMETIK BüRSTEN SET 6 TEILIG AUS HOLZ,0,0,Consumer Goods,Euro,1.3235,9.039505,9.251265,2.0,0.15,0.1
3,150174,KOSMETIK BüRSTEN SET 6 TEILIG AUS HOLZ,0,0,Consumer Goods,Euro,1.2944,8.840752,9.047855,1.0,0.15,0.1
4,144356,KOSMETIK BüRSTEN SET 6 TEILIG AUS HOLZ,0,0,Consumer Goods,Euro,1.2993,8.874219,9.082107,1.0,0.15,0.1


# 2. 异常筛选
## 2.1 drop data where ['bidfee'] >= ['retail']
1. 异常数据，bid fee比retail都高。
    - 有几十条
2. 注意这里的retail的概念
    - 有大概60%的商品可以在amazon上找到，对于这些商品，作者用amazon售价替换了PA网站的建议零售价，构成了*adjusted value*
    - amazon的价格只有PA的79%
    - 另一个数据集的retail的概念是PA网站提供的价格，虽然也提到了amazon售价是偏低的。

In [12]:
# common_auction.shape
# idx_drop = common_auction[(common_auction['k'])>=common_auction['adjvalue']].index
idx_drop = data[(data['k'])>=data['worthupto']].index
data.drop(idx_drop,axis = 0,inplace=True)
print("Drop 掉了 *{}* 行data，他们的bidfee >= retail".format(len(idx_drop)))

Drop 掉了 *0* 行data，他们的bidfee >= retail


## 2.2 desc的问题
- 有无空值: 无


In [13]:
# data_desc = data.item.unique()
# pd.DataFrame(data_desc).to_csv(r'E:\DATA\large_dta\desc.csv', encoding = 'utf-8',index=False)

# 3. 基本规则筛选
## 3.1 only keep pennyauction
- 'pennyauction'这一列是boolean value:
    - 注意pennyauction远远少于非pennyauction
- 'pennyauction'和'fixedprice'有可能同时为1吗:
    - 有的，26行，
    - 这些行应该去掉吗: 去掉，只保留typical pennyauction

In [14]:
tmp1 = data.loc[data['pennyauction']==1]
tmp2 = tmp1.loc[tmp1['fixedprice']==0]
auction_id_1 = tmp2.auctionid
# auction_id_1.shape
print(f"去掉了*{data.shape[0]-auction_id_1.shape[0]}*个data records, 它们不是penny auction")
print(f"现在剩 *{auction_id_1.shape[0]}* 场auction")

去掉了*155696*个data records, 它们不是penny auction
现在剩 *10683* 场auction


## 3.2 k==0(bid inc=0)的data不需要

In [15]:
data_common = pd.merge(auction_id_1,data,on="auctionid",how="left")
data_common_k = data_common[data_common.k != 0]
auction_id_2 = data_common_k.auctionid
print(f"去掉了*{auction_id_1.shape[0]-auction_id_2.shape[0]}*个data records, 它们的bid inc=0")
print(f"现在剩 *{auction_id_2.shape[0]}* 场auction")

去掉了*7*个data records, 它们的bid inc=0
现在剩 *10676* 场auction


In [16]:
auction_id_2.to_csv(PA_auction_id_path,encoding="utf-8",index = False)

## 3.3 对于'item'相同的，对'worthupto'进行统一
1. 一样的商品应该具有一样的价格: 取avg

In [40]:
data_common = pd.merge(auction_id_2,data,on="auctionid",how="left")
items = data_common.item.unique()

In [53]:
data_common_grouped = data_common.groupby(by=['item'],as_index=False)
item_retail = pd.DataFrame(data_common_grouped.worthupto.mean())
item_retail.rename(columns={"worthupto":"retail"},inplace=True)
item_retail['retail'] = np.rint(item_retail.retail)   # 四舍五入

data_new_retail = pd.merge(data_common,item_retail,on=['item'],how="left")
data_new_retail.drop(columns=['worthupto'],inplace=True)
data_new_retail.head(10)


,auctionid,item,pennyauction,fixedprice,type,currency,exchangerate,adjvalue,endtimeperiod,k,y,retail
0,138823,KOSMETIK BüRSTEN SET 6 TEILIG AUS HOLZ,1,0,Consumer Goods,Euro,1.3978,9.546973,10.0,0.01,0.01,10.0
1,175928,20 BIDS GUTSCHEIN,1,0,Consumer Goods,Euro,1.3500,13.500000,18.0,0.01,0.01,14.0
2,176047,20 BIDS GUTSCHEIN,1,0,Consumer Goods,Euro,1.3448,13.448000,18.0,0.01,0.01,14.0
3,176073,20 BIDS GUTSCHEIN,1,0,Consumer Goods,Euro,1.3305,13.305000,15.0,0.01,0.01,14.0
4,176010,20 BIDS GUTSCHEIN,1,0,Consumer Goods,Euro,1.3640,13.639999,32.0,0.01,0.01,14.0
5,175995,20 BIDS GUTSCHEIN,1,0,Consumer Goods,Euro,1.3616,13.616000,16.0,0.01,0.01,14.0
6,175925,20 BIDS GUTSCHEIN,1,0,Consumer Goods,Euro,1.3500,13.500000,5.0,0.01,0.01,14.0
7,180618,20 BIDS GUTSCHEIN,1,0,Consumer Goods,Euro,1.3963,13.962999,118.0,0.01,0.01,14.0
8,176043,20 BIDS GUTSCHEIN,1,0,Consumer Goods,Euro,1.3448,13.448000,6.0,0.01,0.01,14.0
9,176006,20 BIDS GUTSCHEIN,1,0,Consumer Goods,Euro,1.3640,13.639999,42.0,0.01,0.01,14.0


## 3.4 翻译item

In [62]:
data = pd.read_csv(auctions_cleaned_path,encoding="utf-8")
item = pd.DataFrame(data.item.unique())
print(f"一共有auction记录 *{data.shape[0]}* 条，一共有 *{item.shape[0]} 条item*")
item.insert(1,"item_en","")
item.rename(columns={0:"item"},inplace=True)
item.head()
# for i in range()

一共有auction记录 *10676* 条，一共有 *758 条item*


,item,item_en
0,KOSMETIK BüRSTEN SET 6 TEILIG AUS HOLZ,
1,20 BIDS GUTSCHEIN,
2,20 BIDS VOUCHER,
3,50 BIDS GUTSCHEIN,
4,50 FREEBIDS-GUTSCHEIN!,


In [ ]:
import translators as ts
from_language, to_language = 'de', 'en'
for i in tqdm(range(item.shape[0])):
    text_de = item.iloc[i,0]
    text_en = ts.translate_text(text_de, from_language = from_language, to_language = to_language,if_ignore_empty_query=False, if_ignore_limit_of_length=False, limit_of_length=5000)
    item.iloc[i,1] = text_en

print("Done")

In [83]:
item=pd.read_csv(r'E:\DATA\large_dta\items.csv')
print(f"在翻译之前，有 *{item.shape[0]}* 个items，翻译后有 *{len(item.item_en.unique())}* 个items")
# 手动纠正了两个翻译
data_en = pd.merge(data_new_retail,item,on="item",how="left")
data_en.drop(columns=['item'],inplace=True)
data_en.rename(columns={"item_en":"item"},inplace=True)
data_en.head()

在翻译之前，有 *758* 个items，翻译后有 *755* 个items


,auctionid,pennyauction,fixedprice,type,currency,exchangerate,adjvalue,endtimeperiod,k,y,retail,item
0,138823,1,0,Consumer Goods,Euro,1.3978,9.546973,10.0,0.01,0.01,10.0,COSMETICS Brush SET 6 PIECES MADE OF WOOD
1,175928,1,0,Consumer Goods,Euro,1.3500,13.500000,18.0,0.01,0.01,14.0,20 BIDS VOUCHER
2,176047,1,0,Consumer Goods,Euro,1.3448,13.448000,18.0,0.01,0.01,14.0,20 BIDS VOUCHER
3,176073,1,0,Consumer Goods,Euro,1.3305,13.305000,15.0,0.01,0.01,14.0,20 BIDS VOUCHER
4,176010,1,0,Consumer Goods,Euro,1.3640,13.639999,32.0,0.01,0.01,14.0,20 BIDS VOUCHER


## 3.5 endtimeperiod一定是整数

In [84]:
data_en['endtimeperiod'] = np.rint(data_en.endtimeperiod)   # 四舍五入

## 3.6 Save data


In [85]:
data_en.to_csv(auctions_cleaned_path,encoding="utf-8",index=False,header=True)

# 4. 保存settings
## 4.1 对于GT: setting = ['k','y','worthupto']
1. data_key_GT = ['k','y','worthupto']，只需要这3个attrb.就可以计算GT的output，不在乎item是什么

In [86]:
auction_id = pd.read_csv(PA_auction_id_path,encoding="utf-8")   # 规则上合理的auction_id
data = pd.read_csv(auctions_cleaned_path,encoding="utf-8")
assert data.shape[0] == auction_id.shape[0],"wrong!"

In [87]:
unique_setting_GT = ['k','y','retail']
# data_new = pd.merge(auction_id,data,on="auctionid",how="left")

data_auction_GT = data[unique_setting_GT].copy()
settings_GT = data_auction_GT.drop_duplicates(ignore_index=True)
settings_GT.to_csv(settings_GT_path,encoding="utf-8",index=False,header=True)

print(f"当setting = {unique_setting_GT}")
print(f"一共有 *{settings_GT.shape[0]}*场不同的auction")

当setting = ['k', 'y', 'retail']
一共有 *618*场不同的auction


## 4.2 对于NN/ training/ target: ['item',k','y','worthupto']
1. 说明还是存在一个item对应多个auction records

In [88]:

unique_setting_NN = ['item','k','y','retail']
# data_new = pd.merge(auction_id,data,on="auctionid",how="left")
# 把'k'转化成dollar instead of cent
# data_new.k = data_new.k*0.01

data_auction_NN = data[unique_setting_NN].copy()
settings_NN = data_auction_NN.drop_duplicates(ignore_index=True)
settings_NN.to_csv(settings_NN_path,encoding="utf-8",index=False,header=True)

print(f"当setting = {unique_setting_NN}")
print(f"一共有 *{settings_NN.shape[0]}*场不同的auction")

当setting = ['item', 'k', 'y', 'retail']
一共有 *758*场不同的auction
